### Lab Tutorials

## Tutorial 4: Frequency analysis

This tutorial will take a look at dynamic motions and loads using frequency analysis. 

Start by importing the same libraries.

In [ ]:
from analysis import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.max_rows = 20

Next, lets read in the wave data as we did in tutorial 1.

In [ ]:
filename = '../WaveProbe/run8_9.csv'
(read_waves(filename)
 .query('time>100 & time<200')
 .plot(x='time'))

Note the amplitude is around $0.04m$, and (zooming in) the frequency is around $16/20 = 0.8 Hz$

The `analysis.py` file has a function to take the [Fast Fourier Transform (FFT)](https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.fft.html) of a data frame. Let's apply it to a run.

In [ ]:
(read_waves(filename)
 .query('time>100 & time<200')
 .pipe(take_FFT))

The function has converted time to frequency and replaced our signal with its FFT.

Recall that an FFT determines the amplitude of sinusoids at every frequency which add together to reproduce the signal. The frequency bins are determined by the sampling rate and duration of the signal. Let's plot this data.

In [ ]:
(read_waves(filename)
 .query('time>100 & time<200')
 .pipe(take_FFT)
 .query('freq<5 & freq>0')
 .plot(x='freq',logy=True));

This data looks rough on a log scale since very small amplitudes can be seen clearly. However, there is a clear peak at $0.8~Hz$ with amplitude around $0.04~m$, as we guessed from the time trace. However, the FFT shows there are also smaller peaks at $1.6,2.4,3.2~Hz$; these are the high-order _wave harmonics_ we discussed in lecture.

Next, lets repeat this analysis for the wave forcing on the fixed platform. 

In [ ]:
(read_fixed_strain('../FixedStrain/run8.txt')
 .query('time>85')
 .pipe(take_FFT)
 .query('freq<5 & freq>0')
 .plot(x='freq',logy=True));

The two lines have different units, so comparing their relative magnitudes is meaningless. However, the plot clearly shows that __the peak force on the piling occurs at the peak wave frequency__.

Also notice that since we don't have very many cycles, our frequency resolution isn't nearly as good. This is why we wanted as many waves as possible in the experiments. 

How about the motions? Let's take a look.

In [ ]:
(read_qualisys('../Qualisys/run0008_0009_6D.tsv',2)
 .query('time>340 & time<420')
 .pipe(take_FFT)
 .query('freq>0 & freq<5')
 .plot(x='freq',y='q_3',logy=True));

Just as with the forces, the platforms are both heaving at the wave frequency and it's harmonics. 

How should this be extended to the irregular wave case?

In [ ]:
(read_waves('../WaveProbe/run7.csv')
 .query('time>50 & time<350')
 .pipe(take_FFT)
 .query('freq>0 & freq<5')
 .plot(x='freq'));

The spectra looks ok, but it is very noisy. We can't use this to get an RAO.

We can de-noise these using Welch's method of splitting the signal into segments and averaging.

In [ ]:
print(help(take_welch))

(read_waves('../WaveProbe/run7.csv')
 .query('time>50 & time<350')
 .pipe(take_welch,nseg=16)
 .query('freq>0 & freq<5')
 .plot(x='freq'));

Looks __much__ nicer. But beware, this comes at the cost of frequency resolution (just like above). We can write a general function and test it against run 8 that we looked at about.

In [ ]:
def wave_spectra_plot(run,start,stop,nseg=1):
    (read_waves('../WaveProbe/run{}.csv'.format(run))
     .query('time>@start & time<@stop')
     .pipe(take_welch,nseg=nseg)
     .query('freq>0 & freq<5')
     .plot(x='freq',logy=True))
    plt.title('Run{}'.format(run))
    plt.show()
    
wave_spectra_plot(run='8_9',start=100,stop=200,nseg=16)

The nice thing about Welch's method is that it doesn't loose the harmonic peaks, even when de-noising.

Finally, let's look at all three irregular runs.

In [ ]:
windows = pd.DataFrame({'run':  [  3,  7, 10],  # make a DF with the run 
                        'start':[130, 50,130],  # window start time
                        'stop': [500,350,500]}) # and window stop time

for row in windows.itertuples(index=False): #loop through rows. Ugly, I know.
     wave_spectra_plot(row.run,row.start,row.stop,nseg=16)

We can see run 3 is __really__ low amplitude, and it hasn't filled in the whole spectrum. Run 7 and 10 are much higher, and generally look pretty good.